In [1]:
from enum import Enum

In [51]:
def func1(x):
    return x**3 + x**2

In [65]:
def func1_derivative(x):
    return 3 * (x ** 2) + 2 * x

Znajdowanie miejsc zerowych funkcji nieliniowych metodą siecznych

![title](img_vsc/img01.png)

In [56]:
# Stopping conditions for secant_method
class ConditionSecant(Enum):
    ABS = lambda y_i, eps : abs(y_i) < eps
    DISTANCE = lambda x_1, x_2, eps: abs(x_1 - x_2) < eps

In [60]:
def secant_method(f, x_0, x_1, condition, eps, max_iterations):
    
    x_i = x_0
    x_j = x_1 # x_j = x_(i + 1)
    
    for _ in range(max_iterations):
        
        if f(x_i) == f(x_j): # Division by zero, thus no solution
            return None
        
        x_i, x_j = x_j, x_j - ( (x_j - x_i) / (f(x_j) - f(x_i)) ) * f(x_j)
        
        if condition == ConditionSecant.ABS and ConditionSecant.ABS(f(x_j), eps):
            return x_j
        elif condition == ConditionSecant.DISTANCE and ConditionSecant.DISTANCE(x_i, x_j, eps):
            return x_j
    
    print("Exceeded max number of iterations")
    return None # Exceeded number of iterations

In [62]:
print(secant_method(f = func1, x_0 = -0.5, x_1 = 1, condition = ConditionSecant.ABS, eps = 1e-18, max_iterations = 1000))

-1.0


Znajdowanie miejsc zerowych funkcji nieliniowych metodą Newtona-Raphsona

![title](img_vsc/img02.png)

In [63]:
# Stopping conditions for newton_raphson_method
class ConditionNewton(Enum):
    ABS = lambda y_i, eps : abs(y_i) < eps
    DISTANCE = lambda y_i, yd_i, eps: abs(y_i / yd_i) < eps

In [68]:
def newton_raphson_method(f, fd, x_0, condition, eps, max_iterations):
    
    x_i = x_0
    
    for _ in range(max_iterations):
        
        y_i = f(x_i)
        yd_i = fd(x_i)
        
        if yd_i == 0: # Derivative equal to zero
            return None
        
        if condition == ConditionNewton.ABS and ConditionNewton.ABS(f(y_i), eps):
            return x_i
        elif condition == ConditionNewton.DISTANCE and ConditionNewton.DISTANCE(y_i, yd_i, eps):
            return x_i
        
        x_i  -= y_i / yd_i        
        
    return None # Max iterations exceeded

In [83]:
print(newton_raphson_method(f = func1, fd = func1_derivative, x_0 = 0.01, condition = ConditionNewton.DISTANCE, eps = 1e-18, max_iterations = 1000))

1.1212887150498135e-18
